In [4]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 13.6 MB/s eta 0:00:00


In [35]:
# ================================================================
# IMPORT LIBRARY
# ================================================================
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider


# ================================================================
# 1. UNIVERSE & MEMBERSHIP FUNCTIONS
# ================================================================
suhu = np.arange(0, 41, 1)
kelembapan = np.arange(0, 101, 1)
kipas = np.arange(0, 101, 1)

# Suhu (Membership Functions)
suhu_dingin = fuzz.trimf(suhu, [0, 0, 20])
suhu_hangat = fuzz.trimf(suhu, [15, 25, 35])
suhu_panas  = fuzz.trimf(suhu, [30, 40, 40])

# Kelembapan
hum_rendah = fuzz.trimf(kelembapan, [0, 0, 40])
hum_sedang = fuzz.trimf(kelembapan, [30, 50, 70])
hum_tinggi = fuzz.trimf(kelembapan, [60, 100, 100])

# Kipas
kipas_pelan  = fuzz.trimf(kipas, [0, 0, 50])
kipas_sedang = fuzz.trimf(kipas, [30, 50, 80])
kipas_cepat  = fuzz.trimf(kipas, [50, 100, 100])


# ================================================================
# 2. FUNGSI INTERAKTIF MAMDANI
# ================================================================
def fuzzy_mamdani(suhu_val, kelembapan_val):

    # 2.1 Fuzzifikasi
    mu_suhu_dingin = fuzz.interp_membership(suhu, suhu_dingin, suhu_val)
    mu_suhu_hangat = fuzz.interp_membership(suhu, suhu_hangat, suhu_val)
    mu_suhu_panas  = fuzz.interp_membership(suhu, suhu_panas,  suhu_val)

    mu_hum_rendah = fuzz.interp_membership(kelembapan, hum_rendah, kelembapan_val)
    mu_hum_sedang = fuzz.interp_membership(kelembapan, hum_sedang, kelembapan_val)
    mu_hum_tinggi = fuzz.interp_membership(kelembapan, hum_tinggi, kelembapan_val)

    # 2.2 Aturan Fuzzy (Mamdani)
    alpha1 = np.fmax(mu_suhu_dingin, mu_hum_rendah)   # → Kipas pelan
    alpha2 = mu_suhu_hangat                           # → Kipas sedang
    alpha3 = np.fmax(mu_suhu_panas, mu_hum_tinggi)    # → Kipas cepat

    # 2.3 Implifikasi
    pelan_clipped  = np.fmin(alpha1, kipas_pelan)
    sedang_clipped = np.fmin(alpha2, kipas_sedang)
    cepat_clipped  = np.fmin(alpha3, kipas_cepat)

    # 2.4 Agregasi
    aggregated = np.fmax(pelan_clipped,
                    np.fmax(sedang_clipped, cepat_clipped))

    # 2.5 Defuzzifikasi
    centroid = fuzz.defuzz(kipas, aggregated, 'centroid')
    bisector = fuzz.defuzz(kipas, aggregated, 'bisector')
    mom      = fuzz.defuzz(kipas, aggregated, 'mom')

    # 2.6 Visualisasi
    plt.figure(figsize=(8, 4))
    plt.plot(kipas, kipas_pelan, 'b--' ,label="Pelan")
    plt.plot(kipas, kipas_sedang, 'g--' ,label="Sedang")
    plt.plot(kipas, kipas_cepat, 'r--'  ,label="Cepat")
    plt.fill_between(kipas, aggregated, color="orange", alpha=0.4, label="Aggregated Output")

    plt.axvline(centroid, color='k', linestyle='-.', label=f'Centroid = {centroid:.1f}')
    plt.axvline(bisector, color='purple', linestyle=':', label=f'Bisector  : {bisector:.1f}')
    plt.axvline(mom, color='brown', linestyle='--', label=f'MOM       : {mom:.1f}')

    plt.title(f"Suhu = {suhu_val:.1f}°C, Kelembapan = {kelembapan_val:.1f}%")
    plt.xlabel("Kecepatan Kipas (%)")
    plt.ylabel("Derajat Keanggotaan")
    plt.ylim(-0.05, 1.05)
    plt.grid(True)
    plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1))
    plt.show()

    # Print hasil
    print(f"Defuzzifikasi:")
    print(f"Centroid : {centroid:.2f}%")
    print(f"Bisector : {bisector:.2f}%")
    print(f"MOM      : {mom:.2f}%")



# ================================================================
# 3. WIDGET INTERAKTIF
# ================================================================
interact(
    fuzzy_mamdani,
    suhu_val=FloatSlider(value=30, min=0, max=40, step=1, description='Suhu (°C)'),
    kelembapan_val=FloatSlider(value=70, min=0, max=100, step=1, description='Kelembapan (%)')
);


interactive(children=(FloatSlider(value=30.0, description='Suhu (°C)', max=40.0, step=1.0), FloatSlider(value=…